## SubhaloClusters

In [1]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr

In [2]:
%%time
halos = spark.read.parquet('/datasets/uchuu/RockstarExtendedParquet/')

CPU times: user 6.72 ms, sys: 2.47 ms, total: 9.19 ms
Wall time: 43.5 s


Correspondence between the snapshot number and redshift for all 50 directories: http://www.skiesanduniverses.org/resources/Uchuu_snapshot_redshift_scalefactor.txt

Parameters:
- Redshift
- Minimum vpeak
- Minimum and maximum halo mass

In [3]:
redshift = 0.00
min_vpeak = 70

# Number of hosts to evaluate with this range: 4
cmass_min = 2.0e15
cmass_max = 2.03e15

# Number of hosts to evaluate with this range: 1140
#cmass_min = 1.0e15
#cmass_max = 5.0e15

# Number of hosts to evaluate with this range: 211022
#cmass_min = 1.0e14
#cmass_max = 5.0e15


Select HOST halos for `Z=0` (redshift=0) in the Mvir range `cmass_min` - `cmass_max`

In [4]:
host_halos = halos.where((col('redshift') == redshift)
                    & (col('pid') == -1)
                    & (col('Mvir') > cmass_min)
                    & (col('Mvir') < cmass_max))

We extend the selected hosts with the new 'CvirKly' column and we rename columns to avoid naming collisions:

In [5]:
hosts = (host_halos
           .withColumn('CvirKly', expr('Rvir/Rs_Klypin'))
           .withColumnRenamed('x', 'host_x')
           .withColumnRenamed('y', 'host_y')
           .withColumnRenamed('z', 'host_z')
           .withColumnRenamed('Rvir', 'host_Rvir')
           .select('id', 'host_x', 'host_y', 'host_z', 'host_Rvir'))

We select the halos that correspond to satellites (pid != -1) and with a given value of 'Vpeak':

In [6]:
satellites = (halos
           .where((col('redshift') == redshift) & (col('Vpeak') > min_vpeak) & (col('pid') != -1))
           .select('pid', 'x', 'y', 'z', 'Vpeak'))

If you want you can take a reduce random sample of the halos, eg. to use 1% of the data:

    myhalos = (halos
           .where((col('redshift') == redshift) & (col('Vpeak') > min_vpeak) & (col('pid') != -1))
           .select('pid', 'x', 'y', 'z', 'Vpeak')
           .sample(0.01))

Since it is small, we will cache in memory the hosts dataframe:

In [7]:
hosts.cache()

DataFrame[id: bigint, host_x: double, host_y: double, host_z: double, host_Rvir: double]

Number of hosts:

In [8]:
hosts.count()

4

Since hosts is a very reduced subset of data, to calculate the median the simplest way is using `Alternative 4: pandas` to calculate the median (see `analyzing_rockstar_extended_data_with_pyspark` notebook to all the alternatives): 

In [9]:
%%time
hosts_pdf = host_halos.withColumn('CvirKly', expr('Rvir/Rs_Klypin')).select('Mvir', 'Rvir', 'CvirKly').toPandas()

CPU times: user 194 ms, sys: 58.9 ms, total: 253 ms
Wall time: 4.67 s


In [10]:
%%time
Mvir_median_pandas = hosts_pdf['Mvir'].median()
Rvir_median_pandas = hosts_pdf['Rvir'].median()
CvirKly_median_pandas = hosts_pdf['CvirKly'].median()

CPU times: user 690 µs, sys: 0 ns, total: 690 µs
Wall time: 680 µs


In [11]:
print('Properties Mean host:','Mvir median =', Mvir_median_pandas, 'Rvir =', Rvir_median_pandas, 'CvirKly =', CvirKly_median_pandas)

Properties Mean host: Mvir median = 2015500000000000.0 Rvir = 2568.325 CvirKly = 5.595388327119377


Results obtained with VAEX:
- Number of hosts = 4
- Properties Mean host: Mvir median = 2015529411764706.0 Rvir = 2568.3263725490197 CvirKly = 5.600783982528668

### Summary results

In [12]:
subhalos = hosts.join(satellites, hosts.id == satellites.pid, 'left')

In [13]:
result = subhalos.withColumn('radius', 
                             F.sqrt(( F.pow(col('host_x') - col('x'), 2)
                                    + F.pow(col('host_y') - col('y'), 2)
                                    + F.pow(col('host_z') - col('z'), 2)))
                             /(col('host_Rvir')/1000.))

In [14]:
%%time
result.groupBy('id', 'host_Rvir').count().selectExpr('id', 'host_Rvir/1000.0', 'count').show()
# Recordar dividir host_Rvir entre 1000

+---------------+--------------------+-----+
|             id|(host_Rvir / 1000.0)|count|
+---------------+--------------------+-----+
|  3814028194986|             2.57321|  857|
|214679739377433|             2.56777|  734|
|139500640393732|  2.5677600000000003|  728|
|101876720636316|             2.56888|  805|
+---------------+--------------------+-----+

CPU times: user 4.94 ms, sys: 2.37 ms, total: 7.3 ms
Wall time: 20.6 s


In [15]:
%%time
radius_quantiles = result.approxQuantile('radius', [0.025, 0.5, 0.975], 0)

CPU times: user 5.61 ms, sys: 4.81 ms, total: 10.4 ms
Wall time: 16.3 s


In [16]:
%%time
rows = result.agg(F.stddev('radius').alias('std')).collect()
radius_std = rows[0].std

CPU times: user 6.69 ms, sys: 1.47 ms, total: 8.16 ms
Wall time: 14.5 s


In [17]:
print(f'Statistics radius:\n'
      f'\tsigma = {radius_std}\n'
      f'\tquantile_0.025 = {radius_quantiles[0]}\n'
      f'\tmedian = {radius_quantiles[1]}\n'
      f'\tquantile_0.975 = {radius_quantiles[2]}\n')

Statistics radius:
	sigma = 0.22744851346072806
	quantile_0.025 = 0.20744410621018508
	median = 0.6424474540350691
	quantile_0.975 = 0.9838158737789053



## Comparing executing times: VAEX vs Spark

VAEX running on FT vs Spark running on Hadoop3. 

The times taken at the Hadoop3 cluster will vary depending on how many resources are assigned to the application. In this case I run them when compiting with other resource intensive application from other user. When running alone the times are even smaller.

The benchmark was run using the smallest example that computes satellites around only 4 hosts:

Results for vaex stored on `snap-50-subhalo-clusters.o4218594` in FT, in 24 hours the calculation could only finish calculating the first host, so to have an estimation of the total time we assume the the other 3 will take a similar time. Addionally the VAEX calculation will have an additional time to compute the global statistics at the end that is not taken into account but that will increase even further the total time.


| _                         | VAEX                     | Spark                     |
|--------------------------|--------------------------|---------------------------|
| Preliminary computations |  109 min                 | less than 1 min           |
| Finding halos            |  9 hours only first host | less than 2 min ALL hosts |
| Total time               |  more than 38 hours (est)| less than 3 min           |

For the medium case example with 1140 hosts the time needed by Spark is less than 6 minutes.